## Data Scrapping Section 

In [1]:
from time import sleep
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import re
import textwrap
from selenium.webdriver.chrome.service import Service
from time import sleep
from contextlib import contextmanager

In [2]:
def create_options():
    ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    options = Options()
    
    options.add_argument(f"user-agent={ua}")
    options.add_argument("--headless")
    options.add_argument("--disk-cache-dir=/dev/null")
    options.add_argument("--disk-cache-size=0")
    options.add_argument("--no-sandbox")
    options.add_argument("--no-zygote")
    options.add_argument("--disable-setuid-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    preferences = {
        "webrtc.ip_handling_policy": "disable_non_proxied_udp",
        "webrtc.multiple_routes_enabled": False,
        "webrtc.nonproxied_udp_enabled": False,
    }
    
    options.add_experimental_option("prefs", preferences)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    return options
    

#### Function to write to csv file 

In [3]:
## Provide directory path of chromeDriver use '/' instead of '\' while giving the path

def get_chromedriver():
    service = Service(r'F:/My Laptop/Techeron//10. Resume job scrapper 27 Apr 2023/chromedriver.exe')  #f:/My Laptop/Techeron//10. Resume job scrapper 27 Apr 2023
    driver = webdriver.Chrome(service=service,options=create_options())
    return driver


In [4]:
@contextmanager
def open_csv_file(file_path, mode):
    try:
        f = open(file_path, mode, newline='', encoding='utf-8')
        yield f
    finally:
        f.close()

def write_adds(data):
    fields = ['Title', 'Company', 'Location', 'Description', 'Date', 'Salary', 'Qualification']
    file_path = f'naukri_data_{job_profile}.csv'
    with open_csv_file(file_path, 'a') as f:
        writer = csv.writer(f)
        if f.tell() == 0:  # if file is empty then write the header
            writer.writerow(fields)
        for row in data:
            writer.writerow(row)



In [5]:
# Getting data from the website and retunring it in list
def get_data_naukri():
    print(f'Getting Data from')
    driver = get_chromedriver()
    datalist = []
    linklist = []
    global job_profile
    job_profile = input("Enter the job profile you want to search:: ")
    search_pages = int(input("Enter Number of pages you want to search::  "))
    driver.get(f'https://www.naukri.com/{job_profile}-jobs')

    try:
        WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@class = "jobTupleHeader"]')))
    except:
        pass

    driver.find_element(By.XPATH, '//*[contains(@class, "search-bar")]/div').click()

    driver.find_element(By.XPATH, '//input[@class = "suggestor-input "]').clear()
    driver.find_element(By.XPATH, '//input[@class = "suggestor-input "]').send_keys(job_profile)
    driver.find_element(By.XPATH, '//button[contains(@class , "icon-wrapper")]').click()
    WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@class = "jobTupleHeader"]')))
    
    #getting links of all the jobs by visiting pagination
    temp = 1
    while True:
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@class = "jobTupleHeader"]')))
        except:
            pass
        jobs = driver.find_elements(By.XPATH, '//*[@class = "jobTupleHeader"]')
        for job in jobs:
            link = job.find_element(By.XPATH, './/a').get_attribute('href')
            linklist.append(link)

        if(temp <= search_pages):  ## Restricting the number of pages using search_pages change here to decrease the execution time
            sleep(3)
            try:
                driver.find_element(By.XPATH, '//a[@class="fright fs14 btn-secondary br2"]').click() # Pagination 
            except:
                break
            temp += 1                   
        else:
            break
        sleep(3)
        

        for link in linklist:
            driver.get(link)
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@class = "jd-header-title"]')))
            except:
                continue
            try:
                title = driver.find_element(By.XPATH, '//h1[@class="jd-header-title"] | //h1[@class="av-special-heading-tag "]').text
            except:
                title = 'None'
            try:
                company = driver.find_element(By.XPATH,'//div[@class="f14 lh18 alignJ"]//p | //div[@class="jd-header-comp-name"]//a').text
            except:
                company = 'None'
            try:
                location = driver.find_element(By.XPATH, '//*[@class = "loc"]').text
            except:
                location = 'None'
            try:
                description = driver.find_element(By.XPATH, '//*[@class = "dang-inner-html"]').text
            except:
                description = 'None'
            try:
                date_divs = driver.find_elements(By.XPATH, '//div[contains(@class, "jd-stats")] | //div[contains(@class, "sumFoot")]')
                date = [div.find_element(By.XPATH, './span').text for div in date_divs][0][7:]          
            except:
                date = 'None'
            try:
                salary_divs = driver.find_elements(By.XPATH, '//div[@class="salary"] | //div[@class="slide-meta getSalaryRange"]')
                salary = [div.find_element(By.XPATH, './span').text for div in salary_divs][0]
            except:
                salary = 'None'
            try:
                qualification = driver.find_element(By.XPATH, '//*[@class = "exp"]')
                if qualification.text.startswith('Key Skills:'):
                    qualification = qualification.find_element(By.XPATH, './following-sibling::div').text
                else:
                    qualification = qualification.text
            except:
                qualification = 'None'
            data = [title, company, location, description, date, salary, qualification]
            datalist.append(data)
            write_adds(datalist)
            sleep(3)

#### Main function 

In [6]:
if __name__ == '__main__':
    get_data_naukri()

Getting Data from
Enter the job profile you want to search:: PHP Developer
Enter Number of pages you want to search::  2
